In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [15]:
model = YOLO('./model/client_model/best.pt')

## READ FROM VIDEO RECORD

In [14]:
video_path = './dataset/Video/Video_2_30_detik.mp4'
cap = cv2.VideoCapture(video_path)

## READ FROM WEBCAM

In [11]:
cap = cv2.VideoCapture(0) 

## CLASS MAPPING

In [16]:
class_mapping = {0 : "Mobil", 1 : "Motor", 2 : "Plat Nomor"}

In [17]:
ret = True
captured = []
plate = 0

while ret:
    ret, frame = cap.read()
    
    if ret:
        
        new_height = int(frame.shape[0] * 0.3)  # 25% dari tinggi frame asli
        new_width = frame.shape[1]  # Lebar sesuai dengan frame asli

        # Hitung koordinat untuk rectangle
        top_left_x = 0
        top_left_y = frame.shape[0] - new_height - int(frame.shape[0] * 0.1)  # Angkat rectangle ke tengah 20%
        bottom_right_x = frame.shape[1]
        bottom_right_y = top_left_y + new_height
        
        cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 255), thickness=2)
        detected_area = frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
        result = model.track(detected_area, persist=True)
        
        if result is not None and result[0].boxes.id is not None:
            track_ids = result[0].boxes.id.int().cpu().tolist()
            boxes = result[0].boxes.xywh.cpu()
            frame_ = result[0].plot()   
            
            for i, track_id in enumerate(track_ids):
                x1, y1, x2, y2, id, score, class_id = result[0].boxes.data[i].numpy()
                if track_id not in captured and int(class_id) == 2:
                    captured.append(track_id)
                    # Melakukan pemotongan (cropping) frame
                    capture_frame = detected_area[int(y1):int(y2), int(x1):int(x2)]
                    cv2.imwrite(f'./license_predict/{int(track_id) }.jpg', capture_frame)
                    
                # Menggantikan area yang dicrop dengan frame_
                frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x] = frame_
                    
            ## MENAMPILKAN LIVE
            # Mendapatkan dimensi asli frame
            original_height, original_width = frame.shape[:2]

            # Hitung faktor skala untuk menyesuaikan frame agar muat dalam satu layar laptop
            screen_height = 720  # Misalnya, resolusi layar laptop adalah 720p
            screen_width = 1280  # Misalnya, resolusi layar laptop adalah 1280p
            scale_factor = min(screen_height / original_height, screen_width / original_width)

            # Resize frame sesuai dengan faktor skala
            resized_frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor)
            
            cv2.imshow('frame', resized_frame)
            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            print("Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.")
            continue  # Lanjutkan ke frame berikutnya jika hasil tidak tersedia
        
cv2.destroyAllWindows()


0: 160x800 3 Motors, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 800)

0: 160x800 3 Motors, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 800)

0: 160x800 3 Motors, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 800)

0: 160x800 3 Motors, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 2.6ms postprocess per image at shape (1, 3, 160, 800)

0: 160x800 3 Motors, 5.1ms
Speed: 2.0ms preprocess, 5.1ms inference, 0.9ms postprocess per image at shape (1, 3, 160, 800)

0: 160x800 3 Motors, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.6ms postprocess per image at shape (1, 3, 160, 800)

0: 160x800 4 Motors, 6.0ms
Speed: 1.1ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 800)

0: 160x800 4 Motors, 6.5ms
Speed: 0.0ms preprocess, 6.5ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 800)

0: 160x